In [1]:
import numpy as np
import napari
import pandas as pd
from tqdm import tqdm
import zarr
import mFISHwarp.morphology
import mFISHwarp.fft
from skimage.filters import threshold_otsu
from skimage.exposure import match_histograms
import pickle
from scipy import fft
import dask.array as da
import os

In [3]:
# image path
# '/mnt/ampa02_data01/tmurakami/240425_whole_4color_2nd_M037-3pb/fused/fused.n5'
# '/mnt/ampa02_data01/tmurakami/240417_whole_4color_1st_M037-3pb/fused/fused.n5'
data_path = '/mnt/ampa02_data01/tmurakami/240425_whole_4color_2nd_M037-3pb/fused/fused.n5'

voxel_size = (2.0,1.3,1.3)
reference_chan = 3
segment_chan = 2
upper_percentile = 99
lower_percentile = 30
resolution = 3 # s2 is the maximum considering the RAM size.

hann_window_shrink_factors = [1]
# hann_window_shrink_factor = 10

_, ext = os.path.splitext(data_path)


if ext == '.n5': # n5 assume bigstitcher (bigdataviewer) format
    # create Zarr file object
    # load images according to the input parameters.
    img_zarr = zarr.open(store=zarr.N5Store(data_path), mode='r')
    n5_setups = list(img_zarr.keys())
    res_list = list(img_zarr[n5_setups[reference_chan]]['timepoint0'].keys())

    # img_da = da.from_zarr(img_zarr[n5_setups[segment_chan]]['timepoint0'][res_list[resolution]])
    
viewer = napari.Viewer()    

# make mask that covers only tissue to exclude the area outside of the tissue.
img_down_ref = img_zarr[n5_setups[reference_chan]]['timepoint0'][res_list[resolution]][:]
global_thresh = threshold_otsu(img_down_ref)
img_mask = mFISHwarp.morphology.mask_maker(img_down_ref,global_thresh)

img_down = img_zarr[n5_setups[segment_chan]]['timepoint0'][res_list[resolution]][:]
shape = img_down.shape

# make mask in Fourier space to remove low frequency component.
for hann_window_shrink_factor in hann_window_shrink_factors:
    hann_3d = mFISHwarp.fft.create_3d_hann_window(tuple(i//hann_window_shrink_factor for i in shape), shape)
    mask = 1 - hann_3d

    filtered_image = mFISHwarp.fft.fft_filter(img_down, mask)

    lower = np.percentile(img_down[np.where(img_mask)],lower_percentile)
    upper = np.percentile(img_down[np.where(img_mask)],upper_percentile)

    lower_f = np.percentile(filtered_image[np.where(img_mask)],lower_percentile)
    upper_f = np.percentile(filtered_image[np.where(img_mask)],upper_percentile)

    # normalization
    norm_img = (img_down.astype(float)-lower)/(upper-lower)
    norm_img_fft = (filtered_image.astype(float)-lower_f)/(upper_f-lower_f)

    viewer.add_image(norm_img, contrast_limits=[0,1],blending='translucent',name='img')
    viewer.add_image(norm_img_fft, contrast_limits=[0,1],blending='translucent',name='fft')

In [3]:
box_size = 32
z = 85; y = 775; x = 575
# z = 160; y = 584; x = 478
# z = 160; y = 300; x = 233
# z = 100; y = 741; x = 671
# z = 100*2; y = 742*2; x = 638*2
# z = 164; y = 1442; x = 454
# factors = {'s0':1, 's1':2, 's2':4, 's3':8}
factor = 2 ** resolution# factors[resolution]
my_range = (slice(z,z+box_size), slice(y,y+box_size), slice(x,x+box_size))
my_range_in_high = (slice(z*factor,z*factor+box_size*factor),
                    slice(y*factor,y*factor+box_size*factor),
                    slice(x*factor,x*factor+box_size*factor))
matched = match_histograms(norm_img[my_range], norm_img_fft[my_range])

In [4]:
img_high = da.from_zarr(img_zarr[n5_setups[segment_chan]]['timepoint0']['s0'])
block = img_high[my_range_in_high].compute().astype(float)
matched_fft = match_histograms(block, norm_img_fft[my_range])
matched_reg = match_histograms(block, norm_img[my_range])

In [5]:
viewer = napari.Viewer()

viewer.add_image(matched_reg, contrast_limits=[0,1],blending='translucent',name='matched_reg')
viewer.add_image(matched_fft, contrast_limits=[0,1],blending='translucent',name='matched_fft')

<Image layer 'matched_fft' at 0x7fe3494082e0>

In [14]:
viewer = napari.Viewer()

viewer.add_image(norm_img[my_range], contrast_limits=[0,1],blending='translucent',name='img')
viewer.add_image(norm_img_fft[my_range], contrast_limits=[0,1],blending='translucent',name='fft')
viewer.add_image(matched, contrast_limits=[0,1],blending='translucent',name='matched')

<Image layer 'matched' at 0x7efd874e17f0>

In [6]:
#